In [4]:
import pandas as pd
from collections import Counter

def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2) for prob in probs])
def entropy_of_list(a_list):
    cnt=Counter(x for x in a_list)
    num_instance=len(a_list)
    probs=[x/num_instance for x in cnt.values()]
    return entropy(probs)
def information_gain(df,split_attribute_name,target_attribute_name,trace=0):
    df_split=df.groupby(split_attribute_name)
    nobs=len(df.index)
    df_agg_ent=df_split.agg({target_attribute_name:[entropy_of_list,lambda x:len(x)/nobs]})[target_attribute_name]
    df_agg_ent.columns=['Entropy','PropObservation']
    new_entropy=sum(df_agg_ent['Entropy']*df_agg_ent['PropObservation'])
    old_entropy=entropy_of_list(df[target_attribute_name])
    return old_entropy-new_entropy
def id3(df,target_attribute_name,attribute_names,default_class=None):
    cnt=Counter(x for x in df[target_attribute_name])
    print(cnt)
    if len(cnt)==1:
        print(len(cnt))
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class=max(cnt.keys())
        print("Default class : ",default_class)
        gainz=[information_gain(df,attr,target_attribute_name) for attr in attribute_names]
        index_of_max=gainz.index(max(gainz))
        best_attr=attribute_names[index_of_max]
        tree={best_attr:{}}
        print("Best attr: ",best_attr)
        rem_attr=[i for i in attribute_names if i != best_attr]
        for attr_val,data_subset in df.groupby(best_attr):
            subtree=id3(data_subset,target_attribute_name,rem_attr,default_class)
            tree[best_attr][attr_val]=subtree
        return tree
def classify(instance,tree,default=None):
    attribute=next(iter(tree))
    if instance[attribute] in tree[attribute].keys():
        result=tree[attribute][instance[attribute]]
        if isinstance(result,dict):
            return classify(instance,result)
        else:
            return result
    else:
        return default
    
play_tennis=pd.read_csv('../4.csv')
attributes=list(play_tennis.columns)
attributes.remove('PlayTennis')
print("Attribute names: ",attributes)
tree=id3(play_tennis,'PlayTennis',attributes)
from pprint import pprint
pprint(tree)

predicted=pd.read_csv('../4_test.csv')
predicted['predict']=predicted.apply(classify,axis=1,args=(tree,''))
print(predicted[['predict']])

Attribute names:  ['Outlook', 'Temperature', 'Humidity', 'Wind']
Counter({'Yes': 9, 'No': 5})
Default class :  Yes
Best attr:  Outlook
Counter({'Yes': 4})
1
Counter({'Yes': 3, 'No': 2})
Default class :  Yes
Best attr:  Wind
Counter({'No': 2})
1
Counter({'Yes': 3})
1
Counter({'No': 3, 'Yes': 2})
Default class :  Yes
Best attr:  Humidity
Counter({'No': 3})
1
Counter({'Yes': 2})
1
{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
  predict
0     Yes
1      No
2      No
